In [1]:
!pip install wikipedia-api


In [2]:
import numpy as np
import tensorflow as tf
import wikipediaapi
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical


2024-05-21 22:02:20.163798: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-21 22:02:20.163905: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-21 22:02:20.298017: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Fetch Wikipedia article
def fetch_wikipedia_text(topic):
      wiki_wiki = wikipediaapi.Wikipedia(
      language='en',
      extract_format=wikipediaapi.ExtractFormat.WIKI,
      user_agent='my-application/1.0')
      page = wiki_wiki.page(topic)
      return page.text

# Preprocess text
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove special characters
    text = ''.join(c for c in text if c.isalnum() or c in [' ', '\n'])
    return text

In [4]:
topic = "Artificial intelligence"
wikipedia_text = fetch_wikipedia_text(topic)
preprocessed_text = preprocess_text(wikipedia_text)
print(preprocessed_text)


artificial intelligence ai in its broadest sense is intelligence exhibited by machines particularly computer systems it is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and uses learning and intelligence to take actions that maximize their chances of achieving defined goals such machines may be called ais
ai technology is widely used throughout industry government and science some highprofile applications include advanced web search engines eg google search recommendation systems used by youtube amazon and netflix interacting via human speech eg google assistant siri and alexa autonomous vehicles eg waymo generative and creative tools eg chatgpt and ai art and superhuman play and analysis in strategy games eg chess and go however many ai applications are not perceived as ai a lot of cutting edge ai has filtered into general applications often without being called ai because once something become

In [11]:
text = preprocessed_text
# Prepare the tokenizer and encode the words
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
encoded = tokenizer.texts_to_sequences([text])[0]
vocab_size = len(tokenizer.word_index) + 1

# Generate sequences
seq_length = 100
sequences = []
for i in range(seq_length, len(encoded)):
    seq = encoded[i-seq_length:i+1]
    sequences.append(seq)

sequences = np.array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)






In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Define the RNN model
model = Sequential([
    Embedding(vocab_size, 50, input_length=seq_length),
    LSTM(150, return_sequences=True),
    LSTM(100),
    Dense(vocab_size, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=100,callbacks=[tf.keras.callbacks.ReduceLROnPlateau()])


Epoch 1/100
342/342 ━━━━━━━━━━━━━━━━━━━━ 8s 15ms/step - accuracy: 0.0453 - loss: 7.1944 - learning_rate: 0.0010
Epoch 2/100
342/342 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.0492 - loss: 6.5557 - learning_rate: 0.0010
Epoch 3/100
342/342 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.0616 - loss: 6.3878 - learning_rate: 0.0010
Epoch 4/100
342/342 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.0641 - loss: 6.2839 - learning_rate: 0.0010
Epoch 5/100
342/342 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.0695 - loss: 6.2048 - learning_rate: 0.0010
Epoch 6/100
342/342 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.0794 - loss: 6.0793 - learning_rate: 0.0010
Epoch 7/100
342/342 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.0794 - loss: 5.9963 - learning_rate: 0.0010
Epoch 8/100
342/342 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.0905 - loss: 5.8466 - learning_rate: 0.0010
Epoch 9/100
342/342 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.0962 - loss: 5.6987 - learning_rate:

In [15]:
# Generate text function
def generate_text_base_word(model, tokenizer, seed_text, num_words):
    result = []
    in_text = seed_text
    for _ in range(num_words):
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        yhat = model.predict(encoded, verbose=0)
        yhat = np.argmax(yhat, axis=1)
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)

# Example usage
seed_text = "Artificial Intelligence"
generated_text = generate_text_base_word(model, tokenizer, seed_text, 100)
print(seed_text,generated_text)

Artificial Intelligence is the next stage in evolution an idea first proposed by samuel butlers darwin among the situation they are in it is unknown or unobservable and it may not know for certain to ensure fair attribution and compensation for human authors misinformation youtube facebook and others inspired methods and services include gemini formerly bard chatgpt grok claude copilot and llama multimodal gpt models can process different types of data modalities such as images videos sound and text specialized hardware and all findings of aiand that regulators who do using legal reviews by ensure the compliance of military ai with international
